# Code for learning a differentiable weight mask on a transformer

## First, Loading a Model
GPT-2 Medium for now, in Huggingface Transformer library

In [1]:
%load_ext autoreload
%autoreload 2
import os
from cb_utils.models import load_gpt2_weights, load_demo_gpt2, tokenizer
from torch.optim import AdamW
import torch
import pickle
import datasets
from tqdm import tqdm_notebook as tqdm
from itertools import cycle
# from eval import evaluate_model
import plotly.express as px
import pandas as pd

Using device: cuda:0


In [2]:
model = load_demo_gpt2(means=None, weight_masks_attn=True, weight_masks_mlp=True)

In [3]:
param_dict = {}
edge_mask_dict = {}
weight_mask_dict = {}
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)
        param_dict[name] = param
        if 'edge_mask' in name:
            edge_mask_dict[name] = param
        if 'weight_mask' in name:
            weight_mask_dict[name] = param
        
    # type of param is torch.nn.parameter.Parameter

output_mask
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       requires_grad=True)
blocks.0.edge_mask_attentions
Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0',
       requires_grad=True)
blocks.0.edge_mask_mlp
Parameter containing:
tensor([1., 1., 1., 1

In [ ]:
from cb_utils import learn_mask

learn_mask.train_masks()

## Test tasks

In [4]:
from tasks.owt.OWTTask import OWTTask
from tasks.ioi.IOITask import IOITask_old
from tasks.facts.SportsTask import SportsTask

In [5]:
ioi_task = IOITask_old(batch_size=5, tokenizer=tokenizer, template_type="single")
owt_task = OWTTask(batch_size=5, tokenizer=tokenizer, ctx_length=50)

In [55]:
ioi_task = IOITask_old(batch_size=100, tokenizer=tokenizer, template_type="single")

for i in range(10):
    print(ioi_task.get_train_loss(model))
    print(ioi_task.get_test_accuracy(model, check_all_logits=False))
    print(torch.cuda.memory_allocated() / 1024**3)

tensor(0.4578, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.5206, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.4897, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.4724, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.5104, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.5704, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.5012, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.5184, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.4695, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123
tensor(0.4309, grad_fn=<NllLossBackward0>)
1.0
4.886909008026123


In [43]:
owt_task.get_train_loss(model)

token_batch.shape=torch.Size([5, 50]), token_batch=tensor([[18243,   198,   198,  1544,   447,   247,    82,   262,   749,  4950,
           852,    11, 18304,   582,    13,  2080,   257,  2612,   355,  1263,
           355,   262,  8824,    13,   383,   749, 14442,    11, 29012,   582,
            13,   314,   447,   247,   303,  1239,  1775,  9599,   355,  4950,
           355,   339,    13, 16374,  2818,    13,   775,   447,   247,   260],
        [ 1212,  1227,   447,   247,    82,   564,   250, 36772, 49014,   447,
           251,  3033, 12382, 33611,  1754,   286,   383, 31623,   286, 13340,
           287,  6233,  7516,    13, 33611,  1754,  3332,   866,   351, 13340,
          3082, 14001,   284,  2648,   465,  1204,    12,  6511,  1842,   286,
         13340,    11,   262,  5096,  1597,    11,   290,   465, 14066, 21332],
        [15309,   392,  7496,  6069,  2271,   198,   198,   464,   717,   286,
           867, 21000,   380,   292,   287,   262,  6755,  3687,   286,   262,

tensor(4.0969, device='cuda:0', grad_fn=<NllLossBackward0>)

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class IOIPromptsDataset(Dataset):
    def __init__(self, ioi_prompts):
        self.ioi_prompts = ioi_prompts

    def __len__(self):
        return len(self.ioi_prompts)

    def __getitem__(self, idx):
        prompt = self.ioi_prompts[idx]
        text = prompt['text']
        tokens = tokenizer.encode(text)
        last_token_pos = len(tokens) - 1

        return {
            'text': text,
            'tokens': tokens,
            'last_token_pos': last_token_pos
        }

def collate_batch(batch):
    texts = [item['text'] for item in batch]
    tokens = [torch.tensor(item['tokens']) for item in batch]
    last_token_pos = [item['last_token_pos'] for item in batch]

    # Pad the sequences so that they all have the same length
    tokens_padded = pad_sequence(tokens, batch_first=True)

    return {
        'text': texts,
        'tokens': tokens_padded,
        'last_token_pos': torch.tensor(last_token_pos)
    }

ioi_prompts_dataset = IOIPromptsDataset(ioi_prompts)
dataloader = DataLoader(ioi_prompts_dataset, batch_size=3, shuffle=True, collate_fn=collate_batch)

# Pad the sequences in the batch to the same length
batch = next(iter(dataloader))
padded_tokens = pad_sequence(batch['tokens'], batch_first=True)

# Now you can run the whole batched input through the model
output = model(padded_tokens)


NameError: name 'ioi_prompts' is not defined

In [33]:
with open(f"tasks/owt/owt_train.pkl", "rb") as f:
    owt_dataset = pickle.load(f)
# dataset = load_dataset("NeelNanda/pile-10k")['train']

owt_data_loader = DataLoader(owt_dataset, batch_size=3, shuffle=True, num_workers=4, pin_memory=True)

In [34]:
print(len(owt_dataset))

10000


In [35]:
# split owt_dataset into train and test
owt_dataset_train = owt_dataset.select(range(0, 8000))
owt_dataset_test = owt_dataset.select(range(8000, 10000))
# save in owt_train.pkl and owt_test.pkl
with open(f"tasks/owt/owt_train.pkl", "wb") as f:
    pickle.dump(owt_dataset_train, f)
with open(f"tasks/owt/owt_test.pkl", "wb") as f:
    pickle.dump(owt_dataset_test, f)

In [24]:
owt_dataset['tokens']

[[1026,
  318,
  1760,
  11,
  290,
  8948,
  13,
  921,
  460,
  711,
  564,
  250,
  34652,
  2473,
  286,
  262,
  309,
  459,
  6386,
  447,
  251,
  319,
  5565,
  11,
  290,
  319,
  262,
  3992,
  13,
  23911,
  319,
  262,
  3992,
  2499,
  11,
  475,
  345,
  423,
  284,
  29308,
  5021,
  12,
  29332,
  329,
  3084,
  3867,
  290,
  326,
  460,
  307],
 [47934,
  19875,
  2196,
  2625,
  16,
  13,
  15,
  1,
  21004,
  2625,
  48504,
  12,
  23,
  13984,
  29,
  201,
  198,
  27,
  325,
  5154,
  29,
  201,
  198,
  220,
  220,
  220,
  1279,
  3672,
  29,
  5760,
  16,
  3556,
  3672,
  29,
  201,
  198,
  220,
  220,
  220,
  1279,
  11213,
  29,
  12130,
  1153,
  15891,
  1897,
  6826,
  3556,
  11213,
  29],
 [14906,
  2695,
  198,
  198,
  20490,
  4069,
  468,
  257,
  12465,
  2928,
  319,
  14771,
  851,
  517,
  523,
  621,
  749,
  6537,
  851,
  290,
  530,
  286,
  674,
  4094,
  20256,
  318,
  262,
  13542,
  284,
  7558,
  8996,
  290,
  6273,
  6731,
  284,
 

In [22]:
next(iter(owt_data_loader))

{'text': ['Computer systems typically comprise a combination of hardware, such as semiconductors, transistors, chips, and circuit boards, and computer programs. As increasing numbers of smaller and faster transistors can be integrated on a single chip, new processors are designed to use these transistors effectively to increase performance. Currently, many computer designers opt to use the increasing transistor budget to build ever bigger and more complex uni-processors. Alternatively, multiple smaller processor cores can be placed on a single chip, which is beneficial because a single, simple processor core is less complex to design and verify. This results in a less costly and complex verification process, as a once verified module, the processor, is repeated multiple times on a chip.\nTechniques known as multiple logical partitions take advantage of multi-processors. A logically partitioned computer comprises multiple logical partitions that implement virtual computers, which execut

In [28]:
# decode tensor([   48,    39,  5080,    48, 15160,   198, 42246, 31554,  8332, 14155]) of tokens
test_tokens_tensor = torch.Tensor([   48,    39,  5080,    48, 15160,   198, 42246, 31554,  8332, 14155])
print(tokenizer.decode(test_tokens_tensor.tolist()))

QHTwQAust
1968AxDespiteMac
